In [ ]:
!pip install ipython-autotime

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 3.43 s (started: 2021-08-13 01:30:21 +00:00)


# Instala a lib Reranker

In [ ]:
%cd /content/drive/MyDrive/__TCC__Reranker/Reranker/
%pip install -e .

/content/drive/MyDrive/__TCC__Reranker/Reranker
Obtaining file:///content/drive/MyDrive/__TCC__Reranker/Reranker
  Attempting uninstall: reranker
    Found existing installation: reranker 0.0.1
    Can't uninstall 'reranker'. No files were found to uninstall.
  Running setup.py develop for reranker
time: 5.84 s (started: 2021-08-13 01:31:00 +00:00)


In [ ]:
!pwd

/content/drive/My Drive/__TCC__Reranker/Reranker
time: 112 ms (started: 2021-08-13 01:31:09 +00:00)


In [ ]:
!export HF_DATASETS_CACHE="/content/drive/My Drive/_RERANKER_/data"

time: 107 ms (started: 2021-08-09 12:40:13 +00:00)


## Roda o treino do BERT

* flag -m: run library module as a script  
* torch.distributed.launch: 

In [ ]:
%pwd

'/'

time: 6.35 ms (started: 2021-08-13 00:28:26 +00:00)


In [ ]:
%cd ../
%cd /content/drive/MyDrive/__TCC__Reranker/

/content/drive/MyDrive/__TCC__Reranker
/content/drive/MyDrive/__TCC__Reranker
time: 5.4 ms (started: 2021-08-13 00:29:17 +00:00)


In [ ]:
!python -m torch.distributed.launch \
  --nproc_per_node 1 Reranker/run_msmarco/run_marco.py \
  --output_dir checkpoints \
  --model_name_or_path  bert-base-uncased \
  --do_train \
  --save_steps 2000 \
  --train_dir data/minidata/ \
  --max_len 512 \
  --fp16 \
  --per_device_train_batch_size 1 \
  --train_group_size 8 \
  --gradient_accumulation_steps 1 \
  --per_device_eval_batch_size 64 \
  --warmup_ratio 0.1 \
  --weight_decay 0.01 \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --overwrite_output_dir \
  --dataloader_num_workers 8 \

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:164: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  "The module torch.distributed.launch is deprecated "
The module torch.distributed.launch is deprecated and going to be removed in future.Migrate to torch.distributed.run
 Please read local_rank from `os.environ('LOCAL_RANK')` instead.
INFO:torch.distributed.launcher.api:Starting elastic_operator with launch configs:
  entrypoint       : Reranker/run_msmarco/run_marco.py
  min_nodes        : 1
  max_nodes        : 1
  nproc_per_node   : 1
  run_id           : none
  rdzv_backend     : static
  rdzv_endpoint    : 127.0.0.1:29500
  rdzv_configs     : {'rank': 0, 'timeout': 900}
  max_restarts     : 3
  monitor_interval : 5
  log_dir          : None
  metrics_cfg      : {}

INFO:torch.distributed.elastic.agent.server.local_elastic_agent:log directory set to: /tmp/torchelastic_b11k7zm4/none_ewda_vwj
INFO:torch.distributed.elastic.agent.server.a

/root
time: 3.46 ms (started: 2021-08-13 01:31:20 +00:00)


# Inference

# prepare input (acho que sõ tem que fazer uma ũnica vez

In [ ]:
%ls

checkpoints/  colabs/  data/  Reranker/  save_output/
time: 115 ms (started: 2021-08-13 01:33:38 +00:00)


In [ ]:
%cd
%cd /content/drive/MyDrive/__TCC__Reranker/
%mkdir -p save_output
!python Reranker/run_msmarco/topk_text_2_json.py \
  --file data/dev.d100.tsv \
  --save_to save_output/all.json \
  --generate_id_to save_output/ids.tsv \
  --tokenizer bert-base-uncased \
  --truncate 512 \
  --q_truncate -1 

/root
/content/drive/MyDrive/__TCC__Reranker
2021-08-13 01:33:45.882230: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Using custom data configuration default-f3450efa39cfac47
Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-f3450efa39cfac47/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.
queries are not truncated
100% 519300/519300 [21:34<00:00, 401.17it/s]
time: 23min 18s (started: 2021-08-13 01:33:44 +00:00)


In [ ]:
%ls

checkpoints/  colabs/  data/  Reranker/  save_output/  score_saving_dir/
time: 114 ms (started: 2021-08-13 02:01:32 +00:00)


## fazendo o inference no input preparado

In [ ]:
!python Reranker/run_msmarco/run_marco.py \
  --output_dir score_saving_dir \
  --model_name_or_path checkpoints \
  --tokenizer_name bert-base-uncased \
  --do_predict \
  --max_len 512 \
  --fp16 \
  --per_device_eval_batch_size 64 \
  --dataloader_num_workers 8 \
  --pred_path save_output/all.json \
  --pred_id_file  save_output/ids.tsv \
  --rank_score_path score_saving_dir/scores.txt

2021-08-13 02:06:05.811330: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAa

=========== CONFIGURANDO AS COISAS ===========

===========  GETTING DATASET ===========

===========  INITIALIZING TRAINER ===========
Using amp fp16 backend

===========  PREDICTION ===========
***** Running Prediction *****
  Num examples = 519300
  Batch size = 64
 30% 2468/8115 [53:31<2:02:30,  1.30s/it]

# Gerar score MSMarco

In [ ]:
%cd
%cd /content/drive/__TCC__Reranker/__TCC__Reranker
%python /helpers/score_to_marco.py --score_file {path to inference output}

/root
[Errno 2] No such file or directory: '/content/drive/__TCC__Reranker/__TCC__Reranker'
/root
time: 4.7 ms (started: 2021-08-13 00:14:49 +00:00)


UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).
